# Goals: 
- Find out precision, recall, F1 for each tag class in predictions compared to test/val set on a token level and entity level.
- Cannot use this to compare the unmodified test set to the modified test set because the modified test set is too differen

In [1]:
import csv

#Final Model Test
#path_to_true_dataset = r"./datasets/epi_gold-dz/test.tsv"
#path_to_pred_dataset = r'./NER/o14/test_predictions.txt'

#Final Model Evaluation
path_to_true_dataset = r"./datasets/epi_gold-dz/val.tsv"
path_to_pred_dataset = r'./NER/o14/eval_predictions.txt'


## Initial Evaluations
#BioBERT Large initial evaluation
#path_to_true_dataset = r"./datasets/dmis-lab/biobert-large-cased-v1.1/val.tsv"
#path_to_pred_dataset = r'./NER/output2/eval_predictions.txt'

#BioBERT base initial evaluation
#path_to_true_dataset = r"./datasets/dmis-lab/biobert-base-cased-v1.2/val.tsv"
#path_to_pred_dataset = r'./NER/output6/eval_predictions.txt'

#Microsoft PubMedBERT initial evaluation
#path_to_true_dataset = r"./datasets/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract/val.tsv"
#path_to_pred_dataset = r'./NER/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract/eval_predictions.txt'

#Microsoft PubMedBERT+PMC initial evaluation
#path_to_true_dataset = r"./datasets/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/val.tsv"
#path_to_pred_dataset = r'./NER/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/eval_predictions.txt'

#BlueBERT base initial evaluation
#path_to_true_dataset = r"./datasets/bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12/val.tsv"
#path_to_pred_dataset = r'./NER/bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12/eval_predictions.txt'

#BlueBERT large initial evaluation
#path_to_true_dataset = r"./datasets/bionlp/bluebert_pubmed_uncased_L-24_H-1024_A-16/val.tsv"
#path_to_pred_dataset = r'./NER/bionlp/bluebert_pubmed_uncased_L-24_H-1024_A-16/eval_predictions.txt'

## Token-Level Evaluation
- This ignores differences between B-{tag} and I-{tag}

Import ground truth labels

In [2]:
#Load the corrected (trueified) test set data
# DO NOT CHANGE
true_tokens, true_tags= [],[]
with open(path_to_true_dataset,'r') as f:
    reader = csv.reader(f, delimiter="\t", quoting=csv.QUOTE_NONE)
    true_tokens = [row[0] for row in reader if len(row)>1]
    f.seek(0)
    true_tags = [row[1] for row in reader if len(row)>1]
f.close()

In [3]:
for x in range(10):
    print(true_tokens[x],true_tags[x])

Most O
amyotrophic O
lateral O
sclerosis O
( O
ALS O
) O
cases O
are O
considered O


Import the comparison set

In [4]:
#Load the predicted data (final custom dataset)
import csv
with open(path_to_pred_dataset,'r') as f:
    reader = csv.reader(f, delimiter=" ")
    
    #for row in reader:
    #    if len(row)>1:
    #        print(row[0],row[1])
    #    else:
    #        print('')
    pred_tokens = [row[0] for row in reader if len(row)>1]
    f.seek(0)
    pred_tags = [row[1] for row in reader if len(row)>1]
f.close()

In [5]:
for x in range(10):
    print(pred_tokens[x],pred_tags[x])

Most O
amyotrophic O
lateral O
sclerosis O
( O
ALS O
) O
cases O
are O
considered O


In [6]:
print(len(pred_tokens),len(true_tokens))
print(len(pred_tags),len(true_tags))
#Some of the sentences were combined when fixing the test set so there are now different number of sentences and different sizes of sentences

30807 30807
30807 30807


In [7]:
def compare_tag(pred_tag,true_tag):
    comparison_types = ["mismatch","skipped_annotations",
                    "loc_fn","loc_fp",
                    "stat_fn","stat_fp",
                    "date_fn","date_fp",
                    "ethn_fn","ethn_fp",
                    "sex_fn","sex_fp",
                    "dz_fn","dz_fp",
                    "abrv_fn","abrv_fp",
                    "epi_fn","epi_fp"]

    comparison = dict.fromkeys(comparison_types, 0)
    
    if pred_tag != true_tag:
        #This will fn everything, including ones that mismatch from I-LOC -> B-LOC, the others won't
        comparison["mismatch"]+=1
        
        #False positives
        if 'LOC' in pred_tag and 'LOC' not in true_tag:
            comparison["loc_fp"]+=1
        elif 'DIS' in pred_tag and 'DIS' not in true_tag:
            comparison["dz_fp"]+=1
        elif 'ABRV' in pred_tag and 'ABRV' not in true_tag:
            comparison["abrv_fp"]+=1
        elif 'DATE' in pred_tag and 'DATE' not in true_tag:
            comparison["date_fp"]+=1
        elif 'ETHN' in pred_tag and 'ETHN' not in true_tag:
            comparison["ethn_fp"]+=1
        elif 'SEX' in pred_tag and 'SEX' not in true_tag:
            comparison["sex_fp"]+=1
        elif 'STAT' in pred_tag and 'STAT' not in true_tag:
            comparison["stat_fp"]+=1
        elif 'EPI' in pred_tag and 'EPI' not in true_tag:
            comparison["epi_fp"]+=1
        
        #False negatives
        if 'LOC' in true_tag and 'LOC' not in pred_tag:
            comparison["loc_fn"]+=1
        elif 'DIS' in true_tag and 'DIS' not in pred_tag:
            comparison["dz_fn"]+=1
        elif 'ABRV' in true_tag and 'ABRV' not in pred_tag:
            comparison["abrv_fn"]+=1
        elif 'DATE' in true_tag and 'DATE' not in pred_tag:
            comparison["date_fn"]+=1
        elif 'ETHN' in true_tag and 'ETHN' not in pred_tag:
            comparison["ethn_fn"]+=1
        elif 'SEX' in true_tag and 'SEX' not in pred_tag:
            comparison["sex_fn"]+=1
        elif 'STAT' in true_tag and 'STAT' not in pred_tag:
            comparison["stat_fn"]+=1
        elif 'EPI' in true_tag and 'EPI' not in pred_tag:
            comparison["epi_fn"]+=1
        return comparison
    else:
        return comparison

Iterate through all tokens and labels but also accounts for small mismatches (off by one) in the tokens

In [8]:
efficacy_types = ["mismatch","skipped_annotations",
                    "loc_fn","loc_fp",
                    "stat_fn","stat_fp",
                    "date_fn","date_fp",
                    "ethn_fn","ethn_fp",
                    "sex_fn","sex_fp",
                    "dz_fn","dz_fp",
                    "abrv_fn","abrv_fp",
                    "epi_fn","epi_fp"]
efficacy_stats = dict.fromkeys(efficacy_types, 0)

true_idx, pred_idx=0,0

while pred_idx<len(pred_tokens) and true_idx<len(true_tokens):
    if true_tokens[true_idx] != pred_tokens[pred_idx]:
        if true_tokens[true_idx+1] == pred_tokens[pred_idx]:
            true_idx+=1
            #compare
            comparison = compare_tag(pred_tags[pred_idx],true_tags[true_idx])
            for entry in efficacy_types:
                efficacy_stats[entry]+=comparison[entry]
            pred_idx+=1
            true_idx+=1
        elif true_tokens[true_idx] == pred_tokens[pred_idx+1]:
            pred_idx+=1
            #compare
            comparison = compare_tag(pred_tags[pred_idx],true_tags[true_idx])
            for entry in efficacy_types:
                efficacy_stats[entry]+=comparison[entry]
            
            pred_idx+=1
            true_idx+=1
        
        else:
            print('uh oh-->','true_idx:',true_idx,', pred_idx:',pred_idx)
            print(true_tokens[true_idx],pred_tokens[pred_idx])
            efficacy_stats["skipped_annotations"]+=1
            pred_idx+=1
            true_idx+=1
            #skip compare
    else:
        #compare
        comparison = compare_tag(pred_tags[pred_idx],true_tags[true_idx])
        for entry in efficacy_types:
            efficacy_stats[entry]+=comparison[entry]
        
        pred_idx+=1
        true_idx+=1
            
for k,v in efficacy_stats.items():
    print(k,v)

uh oh--> true_idx: 4395 , pred_idx: 4395
α3 [alpha]3
uh oh--> true_idx: 4397 , pred_idx: 4397
α4 [alpha]4
uh oh--> true_idx: 4398 , pred_idx: 4398
α5 [alpha]5
uh oh--> true_idx: 4562 , pred_idx: 4562
β01 [beta]01
uh oh--> true_idx: 5737 , pred_idx: 5737
μm [mu]m
uh oh--> true_idx: 5748 , pred_idx: 5748
μm [mu]m
uh oh--> true_idx: 6226 , pred_idx: 6226
11β0 11[beta]0
uh oh--> true_idx: 6254 , pred_idx: 6254
11β0 11[beta]0
uh oh--> true_idx: 6275 , pred_idx: 6275
μg [mu]g
uh oh--> true_idx: 6297 , pred_idx: 6297
11β0 11[beta]0
uh oh--> true_idx: 22835 , pred_idx: 22835
α [alpha]
mismatch 253
skipped_annotations 11
loc_fn 65
loc_fp 18
stat_fn 68
stat_fp 19
date_fn 2
date_fp 7
ethn_fn 5
ethn_fp 26
sex_fn 1
sex_fp 3
dz_fn 0
dz_fp 0
abrv_fn 0
abrv_fp 0
epi_fn 9
epi_fp 9


In [9]:
#Get the number of true positives
def count_tags(tags):
    tag_types = ['LOC', 'ETHN', 'DATE', 'STAT', 'SEX','DIS','ABRV','EPI']
    tp = dict.fromkeys(tag_types, 0)
    
    for tag in tags:
        for tag_type in tag_types:
            if tag_type in tag:
                tp[tag_type]+=1
                ''' This is equivalent to doing:
                    ```
                    if 'LOC' in tag:
                        l+=1
                    elif 'DIS' in tag:
                        d+=1
                    elif 'ABRV' in tag:
                        a+=1
                    elif 'STAT' in tag:
                        s+=1
                    elif 'EPI' in tag:
                        e+=1
                    ```
                    in the previous code
                '''
    return tp

In [10]:
def get_stats(tp,fp,fn):
    if tp==0:
        return {'precision':0.0,'recall':0.0,'F1':0.0}
    else:
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        F1=(2*precision*recall)/(precision+recall)
        return {'precision':precision,'recall':recall,'F1':F1}

In [11]:
#True Positives
tp = count_tags(true_tags)

#ALl stats
loc_stats = get_stats(tp["LOC"],efficacy_stats["loc_fp"],efficacy_stats["loc_fn"])
stat_stats = get_stats(tp["STAT"],efficacy_stats["stat_fp"],efficacy_stats["stat_fn"])
epi_stats = get_stats(tp["EPI"],efficacy_stats["epi_fp"],efficacy_stats["epi_fn"])
sex_stats = get_stats(tp["SEX"],efficacy_stats["sex_fp"],efficacy_stats["sex_fn"])
ethn_stats = get_stats(tp["ETHN"],efficacy_stats["ethn_fp"],efficacy_stats["ethn_fn"])
date_stats = get_stats(tp["DATE"],efficacy_stats["date_fp"],efficacy_stats["date_fn"])
dz_stats = get_stats(tp["DIS"],efficacy_stats["dz_fp"],efficacy_stats["dz_fn"])
abrv_stats = get_stats(tp["ABRV"],efficacy_stats["abrv_fp"],efficacy_stats["abrv_fn"])

#Overall
overall_tp = sum([val for val in tp.values()])

fp_types = [etype for etype in efficacy_types if 'fp' in etype]
overall_fp = sum([efficacy_stats[fp_type] for fp_type in fp_types])

fn_types = [etype for etype in efficacy_types if 'fn' in etype]
overall_fn = sum([efficacy_stats[fn_type] for fn_type in fn_types])

overall_stats = get_stats(overall_tp, overall_fp, overall_fn)

for k,v in overall_stats.items():
    print('token-level overall',k,v)
print()
for k,v in dz_stats.items():
    print('disease classification',k,v)
print()
for k,v in abrv_stats.items():
    print('abrv classification',k,v)
print()
for k,v in loc_stats.items():
    print('location classification',k,v)
print()
for k,v in date_stats.items():
    print('date classification',k,v)
print()
for k,v in ethn_stats.items():
    print('ethnicity/nationality/race',k,v)
print()
for k,v in sex_stats.items():
    print('biological sex classification',k,v)
print()
for k,v in epi_stats.items():
    print('epi type classification',k,v)
print()
for k,v in stat_stats.items():
    print('epi rate identification',k,v)
print()
print('overall accuracy',1-(efficacy_stats["mismatch"])/(len(true_tags)-efficacy_stats["skipped_annotations"]))

token-level overall precision 0.9213051823416507
token-level overall recall 0.8648648648648649
token-level overall F1 0.892193308550186

disease classification precision 0.0
disease classification recall 0.0
disease classification F1 0.0

abrv classification precision 0.0
abrv classification recall 0.0
abrv classification F1 0.0

location classification precision 0.9357142857142857
location classification recall 0.8012232415902141
location classification F1 0.8632619439868204

date classification precision 0.9615384615384616
date classification recall 0.9887005649717514
date classification F1 0.9749303621169916

ethnicity/nationality/race precision 0.559322033898305
ethnicity/nationality/race recall 0.868421052631579
ethnicity/nationality/race F1 0.6804123711340205

biological sex classification precision 0.9625
biological sex classification recall 0.9871794871794872
biological sex classification F1 0.9746835443037976

epi type classification precision 0.9623430962343096
epi type class

In [12]:
sanity_check=0.0
for k,v in tp.items():
    print(k,v,'Relative weight:',v/overall_tp)
    sanity_check+=v/overall_tp
print('Sanity Check: ',sanity_check)

LOC 262 Relative weight: 0.27291666666666664
ETHN 33 Relative weight: 0.034375
DATE 175 Relative weight: 0.18229166666666666
STAT 183 Relative weight: 0.190625
SEX 77 Relative weight: 0.08020833333333334
DIS 0 Relative weight: 0.0
ABRV 0 Relative weight: 0.0
EPI 230 Relative weight: 0.23958333333333334
Sanity Check:  1.0


In [13]:
overall_tp

960

## Entity-Level Evaluation (with seqeval)

Must re-input the comparison files because this is list of lists of tags, not just list of tags

In [14]:
#Import ground truth, do not change
y_true= []
#'epi_test_setV2-corrected.tsv'
with open(path_to_true_dataset,'r', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter="\t")
    sentences_tags=[]
    for row in reader:
        if len(row)%2==0:
            if len(row)==0:
                y_true.append(sentences_tags.copy())
                sentences_tags.clear()
            else:
                sentences_tags.append(row[1])
f.close()

Import comparison set

In [15]:
#load the predicted data
y_pred= []
with open(path_to_pred_dataset,'r', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=" ")
    sentences_tags=[]
    for row in reader:
        if len(row)%2==0:
            if len(row)==0:
                y_pred.append(sentences_tags.copy())
                sentences_tags.clear()
            else:
                sentences_tags.append(row[1])
f.close()

In [16]:
print(len(y_true),len(y_pred))

1206 1209


In [17]:
#Correct the mismatch
for i in range(min(len(y_pred),len(y_true))):
    if len(y_pred[i]) != len(y_true[i]):
        print(len(y_pred[i]), len(y_pred[i+1]), len(y_true[i]))
        if len(y_pred[i+1])+len(y_pred[i]) == len(y_true[i]):
            y_pred[i]+=y_pred[i+1]
            y_pred.pop(i+1)
            print(len(y_pred[i]), len(y_true[i]))

84 5 89
89 89
84 15 99
99 99
76 7 83
83 83


In [18]:
print(len(y_true),len(y_pred))

1206 1206


In [19]:
from seqeval.metrics import f1_score, precision_score, recall_score
def get_seqevalmetrics(y_true,y_pred):
    return {
            "precision": precision_score(y_true, y_pred),
            "recall": recall_score(y_true, y_pred),
            "f1": f1_score(y_true, y_pred),
        }

In [20]:
print('OVERALL',get_seqevalmetrics(y_true,y_pred))

OVERALL {'precision': 0.8235294117647058, 'recall': 0.8506944444444444, 'f1': 0.8368915456874466}


### Get individual entity types

This will normalize all other entities except for a single one

In [21]:
from copy import deepcopy
#LOCATION
y_true_loc = deepcopy(y_true)
y_pred_loc = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'LOC' not in y_pred[i][j]:
            y_pred_loc[i][j] = 'O'
        if 'LOC' not in y_true[i][j]:
            y_true_loc[i][j] = 'O'

In [22]:
#EPI IDENTIFIER
y_true_epi = deepcopy(y_true)
y_pred_epi = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'EPI' not in y_true[i][j]:
            y_true_epi[i][j] = 'O'
        if 'EPI' not in y_pred[i][j]:
            y_pred_epi[i][j] = 'O'

In [23]:
#EPI Rate
y_true_stat = deepcopy(y_true)
y_pred_stat = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'STAT' not in y_true[i][j]:
            y_true_stat[i][j] = 'O'
        if 'STAT' not in y_pred[i][j]:
            y_pred_stat[i][j] = 'O'

In [24]:
#DATE
y_true_date = deepcopy(y_true)
y_pred_date = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'DATE' not in y_true[i][j]:
            y_true_date[i][j] = 'O'
        if 'DATE' not in y_pred[i][j]:
            y_pred_date[i][j] = 'O'

In [25]:
#SEX
y_true_sex = deepcopy(y_true)
y_pred_sex = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'SEX' not in y_true[i][j]:
            y_true_sex[i][j] = 'O'
        if 'SEX' not in y_pred[i][j]:
            y_pred_sex[i][j] = 'O'

In [26]:
#ETHN
y_true_ethn = deepcopy(y_true)
y_pred_ethn = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'ETHN' not in y_true[i][j]:
            y_true_ethn[i][j] = 'O'
        if 'ETHN' not in y_pred[i][j]:
            y_pred_ethn[i][j] = 'O'

In [27]:
#DIS
y_true_dz = deepcopy(y_true)
y_pred_dz = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'DIS' not in y_true[i][j]:
            y_true_dz[i][j] = 'O'
        if 'DIS' not in y_pred[i][j]:
            y_pred_dz[i][j] = 'O'

In [28]:
#ABRV
y_true_abrv = deepcopy(y_true)
y_pred_abrv = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'ABRV' not in y_true[i][j]:
            y_true_abrv[i][j] = 'O'
        if 'ABRV' not in y_pred[i][j]:
            y_pred_abrv[i][j] = 'O'

In [29]:
#Get all as a single printout
entity_level = {'OVERALL':get_seqevalmetrics(y_true,y_pred),
                'LOCATION':get_seqevalmetrics(y_true_loc,y_pred_loc),
                'EPI IDENTIFIER':get_seqevalmetrics(y_true_epi,y_pred_epi),
                'EPI RATE':get_seqevalmetrics(y_true_stat,y_pred_stat),
                'DATE':get_seqevalmetrics(y_true_date,y_pred_date),
                'SEX':get_seqevalmetrics(y_true_sex,y_pred_sex),
                'ETHN':get_seqevalmetrics(y_true_ethn,y_pred_ethn),
                'DIS':get_seqevalmetrics(y_true_dz,y_pred_dz),
                'ABRV':get_seqevalmetrics(y_true_abrv,y_pred_abrv)}

for typ,metrics in entity_level.items():
    for stat,num in metrics.items():
        print(typ,stat,num)
    print()

/home/wzkariampuzha/.local/lib/python3.6/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/wzkariampuzha/.local/lib/python3.6/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/wzkariampuzha/.local/lib/python3.6/site-packages/seqeval/metrics/v1.py:160: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, 'true nor predicted', 'F-score is', len(true_sum)


OVERALL precision 0.8235294117647058
OVERALL recall 0.8506944444444444
OVERALL f1 0.8368915456874466

LOCATION precision 0.7654320987654321
LOCATION recall 0.7607361963190185
LOCATION f1 0.7630769230769231

EPI IDENTIFIER precision 0.9054726368159204
EPI IDENTIFIER recall 0.9528795811518325
EPI IDENTIFIER f1 0.9285714285714286

EPI RATE precision 0.6875
EPI RATE recall 0.559322033898305
EPI RATE f1 0.6168224299065421

DATE precision 0.8571428571428571
DATE recall 0.8888888888888888
DATE f1 0.8727272727272727

SEX precision 0.9493670886075949
SEX recall 0.9868421052631579
SEX f1 0.967741935483871

ETHN precision 0.5714285714285714
ETHN recall 0.8484848484848485
ETHN f1 0.6829268292682927

DIS precision 0.0
DIS recall 0.0
DIS f1 0.0

ABRV precision 0.0
ABRV recall 0.0
ABRV f1 0.0



diagnose why there was a length mismatch between the predicted and true data

In [30]:
#The sentence where there was the first mismatch (line 7000ish)
x = ["The",
"clinician",
"should",
"be",
"alert",
"to",
"recognize",
"several",
"factors",
"that",
"may",
"maximize",
"renal",
"dysfunction",
"and",
"contribute",
"to",
"the",
"increased",
"incidence",
"of",
"nephrotoxicity",
"associated",
"with",
"these",
"drugs",
",",
"such",
"as",
"intravascular",
"volume",
"depletion",
",",
"the",
"associated",
"use",
"of",
"nonchemotherapeutic",
"nephrotoxic",
"drugs",
"(",
"analgesics",
",",
"antibiotics",
",",
"proton",
"pump",
"inhibitors",
",",
"and",
"bone",
"-",
"targeted",
"therapies",
")",
",",
"radiographic",
"ionic",
"contrast",
"media",
"or",
"radiation",
"therapy",
",",
"urinary",
"tract",
"obstruction",
",",
"and",
"intrinsic",
"renal",
"disease",
"."]

In [31]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-large-cased-v1.1')
len(tokenizer.tokenize(' '.join(x)))

106

Thus the mismatch error is due to tokenizing errors since the max sequence length was 100. Increasing max sequence length to 128 which covers 99.676% of all sentences